In [5]:
!pip install python-terrier
import pyterrier as pt
import pandas as pd
pt.init()

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.7 MB/s eta 0:00:00
  Created wheel for chest: filename=chest-0.2.3-py3-none-any.whl size=7611 sha256=aea5f4856a7f8c74338a3938c0c9bd8716a6daad

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
<ipython-input-5-df7663096205>:4: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [6]:
!rm -rf ./index

# Make sure this dataset exist before running
df = pd.read_csv("cleaned_dataset.csv")

# Make sure the dataset contains columns of 'docno' and 'text'
if 'docno' not in df.columns or 'text' not in df.columns:
    raise ValueError("CSV file must include columns of 'docno' 和 'text' ")

# Transfer dataframe to dictionary form
doc_iter = df.to_dict(orient="records")

# Create Indexer
indexer = pt.IterDictIndexer("./index", meta={'docno': 20, 'text': 4096}, fields=['text'])

# Genrate Index
indexref = indexer.index(doc_iter)

# load index and print it
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 3009
Number of terms: 11620
Number of postings: 164596
Number of fields: 1
Number of tokens: 253171
Field names: [text]
Positions:   false



In [8]:
import gradio as gr
import pandas as pd


# The search function
def search(query):
    if not query.strip():
        return "Please enter a query."
    result_df1 = pt.BatchRetrieve(indexref, wmodel="BM25").search(query)
    return result_df1.head(10) # select first 10 data to show in interface

# Interface
iface = gr.Interface(
    fn=search, # The function use in the interface
    inputs="text", # the input type
    outputs="dataframe", # the output ytpe
    title="News Search Engine",
    description="Enter a query to search for relevant news."
)

iface.launch(share=True) # run interface

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://08c985ccf1cddfc233.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
